In [1]:
import math
import struct
import socket
import random
from threading import Timer

DEFAULT_ADDRESS = "localhost"
DEFAULT_PORT = 5000
INT_FLOAT_DGRAM_LEN = 4
STRING_DGRAM_PAD = 4

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.setblocking(0)

# OSC functions

In [2]:


def send_osc_message(osc_datagram, address, port):
    """Send OSC message via UDP."""
    sock.sendto(osc_datagram, (address, port))


def pad_dgram_four_bytes(dgram):
    """Pad a datagram up to a multiple of 4 bytes."""
    return (dgram + (b'\x00' * (4 - len(dgram) % 4)))


# def string_to_osc(input):
#     """Convert a string into a zero-padded OSC String.
#     The length of the resulting string is always a multiple of 4 bytes.
#     The string ends with 1 to 4 zero-bytes ('\x00')
#     """
#     length = math.ceil((len(input) + 1) / 4.) * 4
#     return struct.pack(">%ds" % length, str(input))


# def touch_1d_message_datagram(pos=0.0):
#     """Construct an osc message with address /touch and one float."""
#     dgram = b''
#     dgram += pad_dgram_four_bytes("/touch".encode('utf-8'))
#     dgram += pad_dgram_four_bytes(",f")
#     dgram += struct.pack('>f', pos)
#     return dgram


def touch_message_datagram(x, y, z):
    """Constructs an OSC messages with three values to trigger a touch sound."""
    dgram = b''
    dgram += pad_dgram_four_bytes("/touch".encode('utf-8'))
    dgram += pad_dgram_four_bytes(",sfsfsf")
    dgram += pad_dgram_four_bytes("/x".encode('utf-8'))
    dgram += struct.pack('>f', x)
    dgram += pad_dgram_four_bytes("/y".encode('utf-8'))
    dgram += struct.pack('>f', y)
    dgram += pad_dgram_four_bytes("/z".encode('utf-8'))
    dgram += struct.pack('>f', z)
    return dgram


def sound_message_datagram(instrument='strings'):
    dgram = b''
    dgram += pad_dgram_four_bytes("/inst".encode('utf-8'))
    dgram += pad_dgram_four_bytes(",s")
    dgram += pad_dgram_four_bytes(instrument.encode('utf-8'))
    return dgram

# High level sound functions.


In [8]:
d = touch_message_datagram(0.0,0.5,1.0)
e = sound_message_datagram()
send_osc_message(d, DEFAULT_ADDRESS, DEFAULT_PORT)
send_osc_message(e, DEFAULT_ADDRESS, DEFAULT_PORT)